# week 3 code DWH

In [5]:
import pandas as pd
import sqlite3 as sql
import warnings
import pyodbc
warnings.simplefilter('ignore')

In [2]:
connection_sales = sql.connect('go_sales_train.sqlite')
connection_crm = sql.connect('go_crm_train.sqlite')
connection_staff = sql.connect('go_staff_train.sqlite')

### verbinden met SDM db

In [6]:

# Verbinden met SDM database
conn_sdm = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=MSI\\SQLEXPRESS;'
    'DATABASE=sdm;'
    'Trusted_Connection=yes;'
)
cursor = conn_sdm.cursor()

# Ophalen van data uit SDM
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = cursor.fetchall()

#denormalisaties
df_order_method = pd.read_sql("SELECT * FROM order_method", conn_sdm)
df_order_header = pd.read_sql("SELECT * FROM order_header", conn_sdm)
df_product_line = pd.read_sql("SELECT * FROM product_line", conn_sdm)
df_product_type = pd.read_sql("SELECT * FROM product_type", conn_sdm)
df_product = pd.read_sql("SELECT * FROM product", conn_sdm)

cursor.close()

conn_sdm.close()

print(tables)


[('age_group',), ('country',), ('course',), ('order_details',), ('order_header',), ('order_method',), ('product',), ('product_line',), ('product_type',), ('retailer',), ('retailer_contact',), ('retailer_headquarters',), ('retailer_segment',), ('retailer_site',), ('retailer_type',), ('return_reason',), ('returned_item',), ('sales_branch',), ('sales_demographic',), ('sales_staff',), ('sales_territory',), ('satisfaction_type',), ('training',)]


### mergen 2 databases

In [7]:
# merge data uit SDM voor denormalisatie
df_merged = pd.merge(df_order_header, df_order_method, on='order_method_code', how='left')
print(df_merged.head())

df_merged = pd.merge(df_product, df_product_type, on='product_type_code', how='left')


Empty DataFrame
Columns: [order_number, retailer_name, retailer_site_code, retailer_contact_code, sales_staff_code, sales_branch_code, order_date, order_method_code, order_method_en]
Index: []


## laden in DWH

In [9]:
conn_dwh = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=MSI\\SQLEXPRESS;'
    'DATABASE=DWH;'
    'Trusted_Connection=yes;'
)
cursor = conn_dwh.cursor()

# Laad de data in een nieuwe DWH-tabel "dim_product_order"
for _, row in df_merged.iterrows():
    cursor.execute("""
        INSERT INTO dim_product_order (product_type_id, product_type_name, product_line_id, product_line_name, order_method_id, order_method_name)
        VALUES (?, ?, ?, ?, ?, ?)
    """, row.product_type_id, row.product_type_name, row.product_line_id, row.product_line_name, row.order_method_id, row.order_method_name)

conn_dwh.commit()
cursor.close()
conn_dwh.close()

print("✅ Data succesvol ingeladen in DWH!")


✅ Data succesvol ingeladen in DWH!
